### AccuWeather 크롤링
- selector 정보 바뀔 수도 있어서 주의 필요
- url로 바로 접근할 방법 (https://www.accuweather.com/ko/kr/seoul/226081/february-weather/226081?year=2021) 형태이므로
    - https://www.accuweather.com/ko/kr/<지역명 영어로>/<지역 코드>/<원하는 월 영어로>-weather/<지역 코드>?year=<원하는 년도>

In [10]:
import pandas as pd
from selenium import webdriver
import re
import time


def weatherfunc(area):
    driver = webdriver.Chrome(executable_path= r'/Users/iyj/Downloads/chromedriver')

    # 원하는 url로 이동
    driver.get("https://www.accuweather.com/ko")
    time.sleep(2)

    # 원하는 지역 입력
    driver.find_element_by_css_selector("div.searchbar-content > form > input").send_keys(area)
    driver.find_element_by_css_selector("div.searchbar-content > svg.icon-search.search-icon").click()

    # time.sleep(3)
    # 해당 지역 이름으로 겁색되는 첫번째 지역 선택
    try:
        driver.find_element_by_css_selector("div.page-column-1 > div.content-module > div.locations-list.content-module > a:nth-child(1)").click()
    except:
        driver.refresh()
        try:
            driver.find_element_by_css_selector("div.page-column-1 > div.content-module > div.locations-list.content-module > a:nth-child(1)").click()
        except:  # 해당 지역이 하나면 클릭할 필요 없으므로
            pass

    # time.sleep(3)
    # 월별 날씨 선택
    try:
        driver.find_element_by_css_selector("div.subnav-items > a[data-gaid='monthly']").click()
    except:
        driver.refresh()
        driver.find_element_by_css_selector("div.page-column-1 > div.content-module > div.locations-list.content-module > a:nth-child(1)").click()
        try:
            driver.find_element_by_css_selector("div.subnav-items > a[data-gaid='monthly']").click()
        except:
            driver.refresh()
            # time.sleep(2)
            driver.find_element_by_css_selector("div.subnav-items > a[data-gaid='monthly']").click()
    n = 0
    wdf = pd.DataFrame({'year':[], 'month':[], 'day':[], 'weather':[], 'weather_info':[], 'high_temp':[], 'low_temp':[]})
    while(n<4):

        year = []
        month = []
        day = []
        weather = []
        weather_info = []
        high_temp = []
        low_temp = []
        # time.sleep(2)

        # 해당 월의 년도 추출
        try:
            ydata = int(''.join(re.findall("\d+",driver.find_elements_by_css_selector("div.monthly-dropdowns > div:nth-child(2) > div.map-dropdown-toggle > h2")[0].text)))
        except:
            driver.refresh()
            # time.sleep(2)
            driver.find_element_by_css_selector("div.subnav-items > a[data-gaid='monthly']").click()
            ydata = int(''.join(re.findall("\d+",driver.find_elements_by_css_selector("div.monthly-dropdowns > div:nth-child(2) > div.map-dropdown-toggle > h2")[0].text)))
        # time.sleep(2)

        # 해당 월 추출
        # 영어로 되어있을 경우 아래 주석 풀고 밑의 코드 주석
        # mdata = driver.find_elements_by_css_selector("div.monthly-dropdowns > div:nth-child(1) > div.map-dropdown-toggle > h2")[0].text
        mdata = int(''.join(re.findall("\d+",driver.find_elements_by_css_selector("div.monthly-dropdowns > div:nth-child(1) > div.map-dropdown-toggle > h2")[0].text)))

        # 해당 월 이미 존재하면 다음 달로 넘기도록
        if mdata in set(wdf.month) and n < 3:
            driver.find_element_by_css_selector("div.two-column-page-content > div.page-column-1 > div.content-module > div.more-cta-links > a.cta-link").click()
            continue

        # 지난 날짜 제외 해당 월의 첫 날부터 마지막 날까지 dataset
        month_data = driver.find_elements_by_css_selector("a.monthly-daypanel:not(.is-past)")
        for i in range(len(month_data)):
            ddata = int(month_data[i].find_element_by_css_selector("div.date").text)
            if ddata == 1 and len(month_data) >= 28 and len(day) < 7: # 해당 월의 시작 일이면 초기화
                year = []
                month = []
                day = []
                weather = []
                weather_info = []
                high_temp = []
                low_temp = []
            if (i > 0 and len(day) > 0) and  ddata < day[-1]: # 다음 월의 데이터 제외하기 위해
                break

            # 날씨 정보
            wdata = int(''.join(re.findall("\d+", month_data[i].find_element_by_css_selector("div.icon-container > svg").get_attribute('src'))))
            wdata_info = month_data[i].find_element_by_css_selector("div.icon-container > svg").get_attribute('alt')
            # 최고 기온
            high = int(''.join(re.findall("\d+", month_data[i].find_element_by_css_selector("div.high").text)))
            # 최저 기온
            low = int(''.join(re.findall("\d+", month_data[i].find_element_by_css_selector("div.low").text)))

            year.append(ydata)
            month.append(mdata)
            day.append(ddata)
            weather.append(wdata)
            weather_info.append(wdata_info)
            high_temp.append(high)
            low_temp.append(low)

        weather_data = {'year':year, 'month':month, 'day':day, 'weather':weather, 'weather_info':weather_info, 'high_temp':high_temp, 'low_temp':low_temp}
        n += 1
        if n < 4:
            try:
                driver.find_element_by_css_selector("div.two-column-page-content > div.page-column-1 > div.content-module > div.more-cta-links > a.cta-link").click()
            except:
                driver.refresh()
                try:
                    driver.find_element_by_css_selector("div.two-column-page-content > div.page-column-1 > div.content-module > div.more-cta-links > a.cta-link").click()
                except:
                    pass

        weather_df = pd.DataFrame(weather_data)
        wdf = pd.concat([wdf,weather_df],ignore_index=True)
        # print(wdf.tail())
    # 날씨 데이터 저장
    wdf.to_excel(r'/Users/iyj/Desktop/weather_data/{}날씨.xlsx'.format(area))
    driver.quit()


In [11]:
areas = ['서울','수원']
for area in areas:
    weatherfunc(area=area)

/var/folders/gy/fn72qmd54376mrszg4b53k_m0000gn/T/ipykernel_2904/625569598.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path= r'/Users/iyj/Downloads/chromedriver')
/var/folders/gy/fn72qmd54376mrszg4b53k_m0000gn/T/ipykernel_2904/625569598.py:15: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector("div.searchbar-content > form > input").send_keys(area)
/var/folders/gy/fn72qmd54376mrszg4b53k_m0000gn/T/ipykernel_2904/625569598.py:16: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector("div.searchbar-content > svg.icon-search.search-icon").click()
/var/folders/gy/fn72qmd54376mrszg4b53k_m0000gn/T/ipykernel_2904/625569598.py:21: DeprecationWarning: find_element_by_cs